### Caixa Bank Demo ###

In [ ]:
%pip install python-dotenv

# Restart the kernel after installing libraries
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

Setting some parameters

In [ ]:
from dotenv import dotenv_values
from urllib.parse import urljoin
from pprint import pprint as pp
import subprocess
import datetime
import json
from pprint import pprint as pp
from graphdatascience import GraphDataScience

## Helper Functions ##

In [ ]:
secret_path = ".secret"

# This function looks for a bearer token and refreshes if it does not exist or is expired
def refresh_token(api_creds, api_base):
    api_endpoint = urljoin(api_base, '/oauth/token')

    curl_cmd = "curl --request POST '{}' --user '{}:{}' --header 'Content-Type: application/x-www-form-urlencoded' --data-urlencode 'grant_type=client_credentials'".format(api_endpoint, api_creds['CLIENT_ID'], api_creds['CLIENT_SECRET'], api_creds['CLIENT_NAME'])
    result = json.loads(subprocess.check_output(curl_cmd, shell=True))
    access_token = result['access_token']
    expires_in = result['expires_in']

    now = datetime.datetime.now()
    expiration = (now + datetime.timedelta(0, expires_in)).isoformat()

    bearer_token = {
        'access_token': access_token,
        'expiration': expiration
    }

    # Save the bearer token to a file
    token_file = secret_path + '/api_bearer_token'
    print('Saving token to file', token_file)
    with open(token_file, "w") as outfile:
        json.dump(bearer_token, outfile, indent=4)

    access_token = bearer_token['access_token']
    return access_token

def deploy_instance(access_token, api_base, instance_name, tenant_id, cloud_provider, region='europe-west1', aura_type='professional-ds', neo4j_version='5', size='16GB'):
    api_endpoint = urljoin(api_base, '/v1/instances')

    instance_details = {}
    print('\nDeployment specifications:')
    print('neo4j_version:', neo4j_version)
    print('region:', region)
    print('memory:', size)
    print('name:', instance_name)
    print('type:', aura_type)
    print('tenant_id', tenant_id)
    print('cloud_provider:', cloud_provider)

    request_body = {
        "version": neo4j_version,
        "region": region,
        "memory": size,
        "name": instance_name,
        "type": aura_type,
        "tenant_id": tenant_id,
        "cloud_provider": cloud_provider
    }

    json_request_body = json.dumps(request_body)

    curl_cmd = "curl -s -X 'POST' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}' -H 'Content-Type: application/json'".format(api_endpoint, access_token)
    curl_cmd += " -d '{}'".format(json_request_body)

    api_response = json.loads(subprocess.check_output(curl_cmd, shell=True))
    print(api_response)
    response_data = api_response['data']

    instance_details['id'] = response_data['id']
    instance_details['name'] = response_data['name']
    instance_details['connection_url'] = response_data['connection_url']
    instance_details['password'] = response_data['password']
    instance_details['username'] = response_data['username']
    instance_details['cloud_provider'] = response_data['cloud_provider']
    instance_details['region'] = response_data['region']
    instance_details['tenant_id'] = response_data['tenant_id']
    instance_details['type'] = response_data['type']

    return instance_details

# Informational functions -- not essential
def tenant_info(access_token, api_base, tenant_id):
    api_endpoint = urljoin(api_base, '/v1/tenants/' + tenant_id)
    tenant_data = {}
    info_cmd = "curl -s -X 'GET' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}'".format(api_endpoint, access_token)
    tenant_info = json.loads(subprocess.check_output(info_cmd, shell=True))['data']

    for item in tenant_info:
        tenant_data[tenant_info['id']] = {}
        tenant_data[tenant_info['id']]['tenant_id'] = tenant_info['id']
        tenant_data[tenant_info['id']]['tenant_name'] = tenant_info['name']
        tenant_data[tenant_info['id']]['instance_configurations'] = list(tenant_info['instance_configurations'])

    return tenant_data

def list_instances(access_token, api_base, tenant_id=None):
    api_endpoint = urljoin(api_base, '/v1/instances')
    aura_instances = {}
    list_cmd = "curl -s -X 'GET' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}'".format(api_endpoint, access_token)

    list_instances = json.loads(subprocess.check_output(list_cmd, shell=True))['data']
    for instance in list_instances:
        aura_instances[instance['id']] = {}
        aura_instances[instance['id']]['instance_id'] = instance['id']
        aura_instances[instance['id']]['instance_name'] = instance['name']
        aura_instances[instance['id']]['cloud_provider'] = instance['cloud_provider']
        aura_instances[instance['id']]['tenant_id'] = instance['tenant_id']

    return aura_instances

def instance_info(access_token, api_base, instance_id):
    api_endpoint = urljoin(api_base, '/v1/instances/' + instance_id)
    instance_data = {}

    info_cmd = "curl -s -X 'GET' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}'".format(api_endpoint, access_token)

    api_response = json.loads(subprocess.check_output(info_cmd, shell=True))

    if 'data' in api_response:
        response_data = api_response['data']
        instance_status = response_data['status']

    return response_data

## Authenticate API ##

In [ ]:
#filename = list(uploaded.keys())[0]
credentials_file = ".secret/aura_ent_api.txt"
print("Credential file:", credentials_file)
aura_creds = dotenv_values(credentials_file)
for item in aura_creds:
  print(item, '=', aura_creds[item])

In [ ]:
api_base = 'https://api.neo4j.io/'
access_token = refresh_token(aura_creds, api_base)
print('access_token:', access_token)

Check to confirm we have access to the Aura tenant


In [ ]:
api_endpoint = urljoin(api_base, '/v1/tenants')
print('api_endpoint:', api_endpoint)

list_cmd = "curl -s -X 'GET' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}'".format(api_endpoint, access_token)
list_tenants = dict(json.loads(subprocess.check_output(list_cmd, shell=True)))['data']

aura_tenants = {}
for item in list_tenants:
    aura_tenants[item['id']] = {}
    aura_tenants[item['id']]['id'] = item['id']
    aura_tenants[item['id']]['name'] = item['name']

print()
print('Aura Tenants:')
print('-------------')
for tenant in aura_tenants:
    print(aura_tenants[tenant])
print()

Now let's take a closer look at our tenant

In [ ]:
tenant_id = "7b9f421f-d9bb-5639-a115-f24dcdf7d17e"
tenant_id = next(iter(aura_tenants))
api_endpoint += '/' + tenant_id
print('api_endpoint:', api_endpoint)

In [ ]:
tenant_details = tenant_info(access_token, api_base, tenant_id)
#pp(tenant_details)
print('tenant_id:', tenant_details[tenant_id]['tenant_id'])
print('tenant_name:', tenant_details[tenant_id]['tenant_name'])
print('available configurations:')
for config in tenant_details[tenant_id]['instance_configurations']:
        print('\t',config['cloud_provider'],config['memory'],config['region'],config['type'],'version:', config['version'])                                

In [ ]:
aura_instances = list_instances(access_token, api_base)
print('aura_instances:')
for instance in aura_instances:
    print(instance, '=', aura_instances[instance])

Let's take a closer look at one of them.

In [ ]:
instance_details = instance_info(access_token, api_base, next(iter(aura_instances)))
print('instance_details:')
print()
for item in instance_details:
    print('\t' + item, '=', instance_details[item])